In [1]:
import pandas as pd
import numpy as np
import feather
import pickle
import re
import sqlite3
import spatialite
import shapely.wkb
import shapely.wkt
from shapely.geometry import *

# optional libs to run other non-core code
from polyfuzz import PolyFuzz
from polyfuzz.models import EditDistance, TFIDF, Embeddings
from flair.embeddings import TransformerWordEmbeddings
import geopandas as gpd

# note pandarallel works well on mac but has issue with windows
# see requirements for windows  - https://github.com/nalepae/pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

pd.options.display.max_columns = None
pd.set_option('display.float_format', lambda x: '%.6f' % x)

# connect to the database
# note: connects to/creates a db file with the name in the quotes if does not exist
db_name='spatial_final.db'
con = sqlite3.connect(db_name)    # for regular SQL
spatcon = spatialite.connect(db_name)    # for spatial SQL
cur = con.cursor()


INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
print(spatcon.execute('SELECT spatialite_version()').fetchone()[0])

5.0.1


## Loading GeoDataFrame to Spatialite

Geopandas has no easy to_sql command only to postgis/postgres we follow the steps in this link to load our data to spatialite:
https://www.giacomodebidda.com/posts/export-a-geodataframe-to-spatialite/

In [3]:
lion = gpd.read_file("Data/LION/LION.shp")

# note some address contain '-' we only want the portion before the dash

lion['l_lowadd'] = lion['LLo_Hyphen'].str.split('-').str[0].astype(float)
lion['l_highadd'] = lion['LHi_Hyphen'].str.split('-').str[0].astype(float)
lion['r_lowadd'] = lion['RLo_Hyphen'].str.split('-').str[0].astype(float)
lion['r_highadd'] = lion['RHi_Hyphen'].str.split('-').str[0].astype(float)

# get combined lower and upper limit for street segment

lion['r_lowadd'].replace({'0':np.nan},inplace=True)
lion['l_lowadd'].replace({'0':np.nan},inplace=True)
lion['c_lowadd']= lion[['l_lowadd','r_lowadd']].min(axis=1,skipna=True)
lion['c_highadd']=lion[['l_highadd','r_highadd']].max(axis=1)
# lion['geometry'] = lion.apply(lambda x: shapely.wkb.dumps(x.geometry), axis=1)

lion.head(5)

,OBJECTID,Street,SAFStreetN,FeatureTyp,SegmentTyp,IncExFlag,RB_Layer,NonPed,TrafDir,TrafSrc,SpecAddr,FaceCode,SeqNum,StreetCode,SAFStreetC,LGC1,LGC2,LGC3,LGC4,LGC5,LGC6,LGC7,LGC8,LGC9,BOE_LGC,SegmentID,SegCount,LocStatus,LZip,RZip,LBoro,RBoro,L_CD,R_CD,LATOMICPOL,RATOMICPOL,LCT2010,LCT2010Suf,RCT2010,RCT2010Suf,LCB2010,LCB2010Suf,RCB2010,RCB2010Suf,LCT2000,LCT2000Suf,RCT2000,RCT2000Suf,LCB2000,LCB2000Suf,RCB2000,RCB2000Suf,LCT1990,LCT1990Suf,RCT1990,RCT1990Suf,LAssmDist,LElectDist,RAssmDist,RElectDist,SplitElect,LSchlDist,RSchlDist,SplitSchl,LSubSect,RSubSect,SanDistInd,MapFrom,MapTo,BoroBndry,MH_RI_Flag,XFrom,YFrom,XTo,YTo,ArcCenterX,ArcCenterY,CurveFlag,Radius,NodeIDFrom,NodeIDTo,NodeLevelF,NodeLevelT,ConParity,Twisted,RW_TYPE,PhysicalID,GenericID,NYPDID,FDNYID,LBlockFace,RBlockFace,LegacyID,Status,StreetWidt,StreetWi_1,StreetWi_2,BikeLane,BIKE_TRAFD,ACTIVE_FLA,POSTED_SPE,Snow_Prior,Number_Tra,Number_Par,Number_Tot,Carto_Disp,FCC,ROW_Type,LLo_Hyphen,LHi_Hyphen,RLo_Hyphen,RHi_Hyphen,FromLeft,ToLeft,FromRight,ToRight,Join_ID,L_PD_Servi,R_PD_Servi,TRUCK_ROUT,Shape__Len,geometry,l_lowadd,l_highadd,r_lowadd,r_highadd,c_lowadd,c_highadd
0,1,EAST 168 STREET,None,0,U,None,B,None,T,DOT,None,2510,03070,226700,None,01,None,None,None,None,None,None,None,None,1,0078126,1,X,10456,10456,2.000000,2.000000,203,203,402,101,149,None,185,None,3001,None,2000,None,149,None,137,None,4000,None,1000,None,149,None,137,None,79,040,79,040,None,09,09,None,1B,1B,None,3D,3D,None,None,1010964,241812,1011265,241555,0,0,None,0,0047740,9045677,M,M,None,None,1,35231.000000,30694.000000,None,None,1422600653,1422602017,0078126,2,34.000000,34.000000,None,None,None,None,25,S,2,None,4,None,None,None,599,699,596,716,599,699,596,716,2251001000000,None,None,None,396.030947,"LINESTRING (-73.90347 40.83036, -73.90238 40.8...",599.000000,699.000000,596.000000,716.000000,596.000000,716.000000
1,2,WEST 192 STREET,None,0,U,None,B,None,A,DOT,None,7984,00040,274810,None,01,None,None,None,None,None,None,None,None,1,0079796,1,None,10468,10468,2.000000,2.000000,207,207,302,104,265,None,265,None,2000,None,1004,None,265,None,265,None,3001,None,1003,None,265,None,265,None,78,045,78,059,None,10,10,None,1A,1A,None,3C,3C,None,None,1011577,255024,1011335,255164,0,0,None,0,0048679,0048678,M,M,None,None,1,35248.000000,30711.000000,None,None,1522607129,1522607721,0079796,2,30.000000,30.000000,None,None,None,None,25,S,1,None,3,None,None,None,58,98,63,99,58,98,63,99,2798401000000,None,None,None,279.360514,"LINESTRING (-73.90120 40.86662, -73.90207 40.8...",58.000000,98.000000,63.000000,99.000000,58.000000,99.000000
2,3,UNION AVENUE,None,0,U,None,B,None,W,DOT,None,7280,00130,270420,None,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2.000000,2.000000,203,203,402,401,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,046,79,026,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252.000000,30715.000000,None,None,1422603726,1422604132,0077356,2,34.000000,34.000000,None,None,None,None,25,S,1,None,3,None,None,None,1017,1079,1016,1084,1017,1079,1016,1084,2728001000000,None,None,None,618.327133,"LINESTRING (-73.90118 40.82440, -73.90051 40.8...",1017.000000,1079.000000,1016.000000,1084.000000,1016.000000,1084.000000
3,4,UNION AVENUE,BEHAGEN PLAYGROUND COMFORT STA,0,U,None,B,None,W,DOT,X,7280,00130,270420,212795,01,None,None,None,None,None,None,None,None,1,0077356,4,X,10459,10459,2.000000,2.000000,203,203,402,401,135,None,131,None,2000,None,3006,None,135,None,131,None,4000,None,4001,None,135,None,131,None,79,046,79,026,None,12,12,None,1A,1A,None,6C,6C,None,None,1011601,239640,1011786,240230,0,0,None,0,0047288,0047822,M,M,None,None,1,35252.000000,30715.000000,None,None,1422603726,1422604132,0077356,2,34.000000,34.000000,None,None,None,None,25,S,1,None,3,None,None,None,None,None,None,None,0,0,0,0,21279502000000X,None,None,None,618.327133,"LINESTRING (-73.90118 40.82440

In [4]:
# check geometry types

lion.geometry.type.unique()

array(['LineString'], dtype=object)

### Step 1: Create Table with NO geospatial features

Spatialite requires you to do it in this order

In [5]:
# First create database table with NO geospatial features

lion.drop(['geometry'],axis=1).to_sql('LION',con,if_exists='replace',index=False)

### Step 2: Add the Geometry Column to the Table

In [6]:
# Then add new column to store geometry

con.enable_load_extension(True)
con.load_extension('mod_spatialite')
con.execute('SELECT InitSpatialMetaData(1);')
con.execute(
    '''
    SELECT AddGeometryColumn('lion','wkb_geometry',4326,'LINESTRING',2);
    '''
)

### Step 3: Convert Geometry to WKT or WKB format

In [7]:
# Convert each shapely geometry into WKT representation

records = [
    {'OBJECTID': lion['OBJECTID'].iloc[i],'wkb':shapely.wkt.dumps(lion['geometry'].iloc[i])}
    for i in range(lion.shape[0])
]

In [8]:
# Create tuple of tuples for query parameter (for batch update with executemany)

tuples = tuple((d['wkb'],d['OBJECTID'].astype(str)) for d in records)
tuples

(('LINESTRING (-73.9034682815071022 40.8303620794707030, -73.9023814757184994 40.8296549103044981)',
  '1'),
 ('LINESTRING (-73.9012021014520002 40.8666213684178032, -73.9020747884371048 40.8670073957739035)',
  '2'),
 ('LINESTRING (-73.9011781129234038 40.8243971910923023, -73.9005068263005001 40.8260159042595987)',
  '3'),
 ('LINESTRING (-73.9011781129234038 40.8243971910923023, -73.9005068263005001 40.8260159042595987)',
  '4'),
 ('LINESTRING (-73.9011781129234038 40.8243971910923023, -73.9005068263005001 40.8260159042595987)',
  '5'),
 ('LINESTRING (-73.9011781129234038 40.8243971910923023, -73.9005068263005001 40.8260159042595987)',
  '6'),
 ('LINESTRING (-73.9069580625364040 40.8936162304320021, -73.9069573857019009 40.8954559390458030)',
  '7'),
 ('LINESTRING (-73.9070665288923010 40.8992824378204034, -73.9071234105827983 40.8992994127918976)',
  '8'),
 ('LINESTRING (-73.9053850046466039 40.8397772683225000, -73.9045463781777983 40.8395062490998981)',
  '9'),
 ('LINESTRING (-73.

### Step 4: Declare an Index

Updating 100 records took 4sec
Updating 1000 records took 42secs

Looks like O(n) - 230k records will take ~3hrs (note: speedcam data has 7mn rows) - since we are updating specific records we should declare an index to speed things up and harness the power of SQL

In [9]:
con.execute("CREATE INDEX OBJECTID ON LION(OBJECTID)")

In [10]:
%%time

with sqlite3.connect(db_name) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.executemany(
        """
        UPDATE LION
        SET wkb_geometry=ST_GeomFromText(? , 4326)
        WHERE lion.OBJECTID = ?;
        """, tuples
    )

CPU times: user 10.4 s, sys: 11.8 s, total: 22.2 s
Wall time: 22.7 s


### Step 5: Create a Spatial Index to use some of the spatial functions like VirtualKNN

In [11]:
spatcon.execute("SELECT CreateSpatialIndex('LION','wkb_geometry')")

### Run a test query to ensure it is set up right

In [12]:
%%time

# query should return line (fid) closest to the point in the query

query='''
Select * FROM knn
WHERE f_table_name = 'LION' 
AND ref_geometry = MakePoint(-73.9721071000000023, 40.6331358999999992,4326)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)
test



CPU times: user 12.5 ms, sys: 5.45 ms, total: 18 ms
Wall time: 123 ms


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00g\xad\xb2\x007~R\xc0...,1,1,40889,0.938217


## Load speedcam data into Spatialite 

Geocode data that has only lat lon - use KNN to find nearest line segment
Run the same steps as above

In [13]:
speedcam = pd.read_feather('Data/tickets/speedred_latlon.feather')

In [14]:
# create a loc_id based on unique lat-lon combinations in the dataset - to use later for joining

speedcam['lat-lon']=speedcam['lon'].astype(str)+speedcam['lat'].astype(str)
loc_dict = dict(zip(speedcam['lat-lon'].unique().tolist(),range(len(speedcam['lat-lon'].unique()))))
speedcam['loc_id']=speedcam['lat-lon'].map(loc_dict)

In [15]:
speedcam.head()

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,Street Code2,Street Code3,Vehicle Expiration Date,Violation Location,Violation Precinct,Issuer Precinct,Issuer Code,Issuer Command,Issuer Squad,Violation Time,Time First Observed,Violation County,Violation In Front Of Or Opposite,House Number,Street Name,Intersecting Street,Date First Observed,Law Section,Sub Division,Violation Legal Code,Days Parking In Effect,From Hours In Effect,To Hours In Effect,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation,StreetNameGeo,lat,lon,lat-lon,loc_id
0,5109306230,88009,NY,MED,01/01/2020 12:00:00 AM,7,SUBN,JEEP,V,0,0,0,0,NaN,0,0,0,None,nan,1201P,None,BK,None,None,SB OCEAN PKWY @ 18TH,AVE,0,1111,D,T,None,None,None,GY,nan,2018,nan,0,nan,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN,SB OCEAN PKWY @ 18TH @ AVE,40.633136,-73.972107,-73.972107140.6331359,0
1,5109308240,LCH6921,PA,PAS,01/01/2020 12:00:00 AM,7,SW,CHEVR,V,0,0,0,0,NaN,0,0,0,None,nan,0601P,None,BK,None,None,NB OCEAN PKWY @ 18TH,AVE,0,1111,D,T,None,None,None,None,nan,2008,nan,0,nan,FAILURE TO STOP AT RED LIGHT,NaN,NaN,NaN,NB OCEAN PKWY @ 18TH @ AVE,40.633136,-73.972107,-73.972107140.6331359,0
2,4675196068,5N37E9,TN,PAS,01/02/2020 12:00:00 AM,36,None,CHRYS,V,0,0,0,0,NaN,0,0,0,None,nan,1232P,None,BK,None,None,EB LINDEN BLVD @ HEM,LOCK ST,0,1180,B,T,None,None,None,WHI,nan,2019,nan,0,nan,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN,EB LINDEN BLVD @ HEM @ LOCK ST,40.668669,-73.868294,-73.8682939999999840.6686685,1
3,4675150573,AWA7408,WA,PAS,01/02/2020 12:00:00 AM,36,SD,TOYOT,V,0,0,0,0,NaN,0,0,0,None,nan,0623A,None,BK,None,None,EB 65TH ST @ 16TH AV,E,0,1180,B,T,None,None,None,None,nan,2011,nan,0,nan,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN,EB 65TH ST @ 16TH AV @ E,38.552879,-121.427041,-121.4270409999999938.552879,2
4,4675150536,AS50483,CT,PAS,01/02/2020 12:00:00 AM,36,SD,TOYOT,V,0,0,0,0,NaN,0,0,0,None,nan,0623A,None,BK,None,None,NB AVE U @ PEARSON S,T,0,1180,B,T,None,None,None,None,nan,2016,nan,0,nan,PHTO SCHOOL ZN SPEED VIOLATION,NaN,NaN,NaN,NB AVE U @ PEARSON S @ T,40.612571,-73.918247,-73.91824740.612570899999994,3


In [16]:
%%time


# create table in datebase - 7mn data points takes awhile to load

speedcam.to_sql('speedcam',con,if_exists='replace',index=False)


/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


CPU times: user 2min 38s, sys: 23.5 s, total: 3min 2s
Wall time: 3min 12s


In [17]:
# Then add new column to store geometry

con.enable_load_extension(True)
con.load_extension('mod_spatialite')
con.execute('SELECT InitSpatialMetaData(1);')
con.execute(
    '''
    SELECT AddGeometryColumn('speedcam','wkb_geometry',4326,'POINT',2);
    '''
)

In [18]:
# convert dataframe to GeoDataFrame to get Point Geom from Lat Lon columns

gdf = gpd.GeoDataFrame(speedcam,geometry=gpd.points_from_xy(speedcam.lon,speedcam.lat))

In [19]:
%%time

# Convert each shapely geometry into WKT representation

records = [
    {'Summons Number': gdf['Summons Number'].iloc[i],'wkb':shapely.wkt.dumps(gdf['geometry'].iloc[i])}
    for i in range(gdf.shape[0])
]

CPU times: user 5min 21s, sys: 3.94 s, total: 5min 25s
Wall time: 5min 26s


In [20]:
# Create tuple of tuples for query parameter (for batch update with executemany)

tuples = tuple((d['wkb'],d['Summons Number'].astype(str)) for d in records)

In [81]:
# with open('tuple.pickle','wb') as f:
#     pickle.dump(tuples,f)

In [3]:
# with open('tuple.pickle','rb') as f:
#     tuples = pickle.load(f)

In [21]:
# create index to improve Update speeds

con.execute("CREATE INDEX  loc_id ON speedcam(loc_id)")
con.execute("CREATE INDEX  `Summons Number` ON speedcam(`Summons Number`)")

In [23]:
%%time


# update geometry


with sqlite3.connect(db_name) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.executemany(
        """
        UPDATE speedcam
        SET wkb_geometry=ST_PointFromText(? , 4326)
        WHERE speedcam.`Summons Number` = ?;
        """, tuples
    )



CPU times: user 3min 54s, sys: 5min 23s, total: 9min 18s
Wall time: 9min 51s


In [24]:
# Create a spatial index
spatcon.execute("SELECT CreateSpatialIndex('speedcam','wkb_geometry')")

In [25]:
%%time

# check to ensure that geoms loaded properly
# also to time a distance formula on 7mn rows

query='''
Select `Summons Number`, Distance(PointFromText('POINT(-73.9215944000000036 40.8278258999999935)'),wkb_geometry) as distance,wkb_geometry
FROM speedcam
Order by distance
'''

test=pd.read_sql_query(query,spatcon)
test[~test['distance'].isnull()]


CPU times: user 33.8 s, sys: 13.6 s, total: 47.4 s
Wall time: 1min 57s


,Summons Number,distance,wkb_geometry
146199,5109306930,0.000000,b'\x00\x01\xe6\x10\x00\x00P\x0b\x14g\xfbzR\xc0...
146200,5109304348,0.000000,b'\x00\x01\xe6\x10\x00\x00P\x0b\x14g\xfbzR\xc0...
146201,5109309140,0.000000,b'\x00\x01\xe6\x10\x00\x00P\x0b\x14g\xfbzR\xc0...
146202,5109307143,0.000000,b'\x00\x01\xe6\x10\x00\x00P\x0b\x14g\xfbzR\xc0...
146203,5109306941,0.000000,b'\x00\x01\xe6\x10\x00\x00P\x0b\x14g\xfbzR\xc0...
...,...,...,...
7345258,4672239111,162.757373,b'\x00\x01\xe6\x10\x00\x00f\xd8(\xeb\xb7\xf6U@...
7345259,4672231768,162.757373,b'\x00\x01\xe6\x10\x00\x00f\xd8(\xeb\xb7\xf6U@...
7345260,4672231641,162.757373,b'\x00\x01\xe6\x10\x00\x00f\xd8(\xeb\xb7\xf6U@...
7345261,4672229749,162.757373,b'\x00\x01\xe6\x10\x00\x00f\xd8(\xeb\xb7\xf6U@...


In [27]:
%%time

#check to ensure KNN is working

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'LION'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)
test.head(100)



CPU times: user 5.98 s, sys: 377 ms, total: 6.35 s
Wall time: 7.61 s


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,lion,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,9666,31.098359


## Create a new table from Unique Lat-Lon Points
Recover geometry and create a spatial index

In [28]:
%%time

spatcon.execute('DROP TABLE IF EXISTS speedcam_points')
spatcon.execute(
    '''
    CREATE TABLE IF NOT EXISTS speedcam_points AS
    Select DISTINCT loc_id,wkb_geometry, count(*) as count
    FROM speedcam
    Group by loc_id
    Order by count DESC
    '''
)
spatcon.execute("SELECT RecoverGeometryColumn('speedcam_points','wkb_geometry',4326,'POINT',2)")
spatcon.execute("SELECT CreateSpatialIndex('speedcam_points','wkb_geometry')")

CPU times: user 553 ms, sys: 162 ms, total: 714 ms
Wall time: 2.24 s


In [3]:
%%time

# check to ensure spatial index is loaded properly on speedcam_points

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'speedcam_points'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)

test

CPU times: user 66 ms, sys: 17.7 ms, total: 83.6 ms
Wall time: 138 ms


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,speedcam_points,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,245,0.000000


## Run KNN on Speedcam Locations to get neartest segment

In [4]:
%%time

query='''
Select k.*,p.loc_id  
FROM knn k, speedcam_points p
WHERE f_table_name = 'LION'
AND ref_geometry = p.wkb_geometry
'''

speedcam_knn=pd.read_sql_query(query,spatcon)
speedcam_knn=speedcam_knn[speedcam_knn['pos']==1][['fid','distance','loc_id']]

CPU times: user 13min 15s, sys: 29.8 s, total: 13min 45s
Wall time: 13min 45s


In [5]:
# create new table
speedcam_knn.to_sql('speedcam_knn',con,if_exists='replace',index=False)

In [6]:
%%time

con.execute('DROP TABLE IF EXISTS speedcam_matched')
con.execute(
    '''
    CREATE TABLE IF NOT EXISTS speedcam_matched AS
    Select a.`Summons Number`,a.`Violation Code`,a.`Issue Date`,a.`Violation Time`,b.*
    FROM speedcam a
    LEFT OUTER JOIN speedcam_knn b
    ON a.loc_id=b.loc_id
    '''
)

CPU times: user 6.76 s, sys: 7.64 s, total: 14.4 s
Wall time: 36.7 s


In [7]:
%%time

con.execute('DROP TABLE IF EXISTS speedcam_final')
con.execute(
    '''
    CREATE TABLE IF NOT EXISTS speedcam_final AS
    Select a.`Summons Number`,a.`Violation Code`,a.`Issue Date`,a.`Violation Time`,
    b.OBJECTID,b.Street,b.FeatureTyp,b.SegmentTyp,b.NonPed,b.TrafDir,b.LocStatus,b.LZip,b.RZip,b.LBoro,b.RBoro,
    b.L_CD,b.R_CD,b.CurveFlag,b.Radius,b.RW_Type,b.PhysicalID,b.StreetWidt,b.BikeLane,b.BIKE_Trafd,b.Number_Tra,
    b.Number_Par,b.Number_Tot,b.Posted_Spe,b.Truck_Rout
    FROM speedcam_matched a
    LEFT OUTER JOIN LION b
    ON a.fid=b.OBJECTID
    '''
)

CPU times: user 19.6 s, sys: 12.4 s, total: 32 s
Wall time: 33.5 s


In [8]:
%%time

query='''
Select *
FROM speedcam_final
'''

test=pd.read_sql_query(query,spatcon)

CPU times: user 1min 22s, sys: 1min 2s, total: 2min 24s
Wall time: 2min 50s


In [9]:
test.to_feather('Data/tickets/speedcam_matched.feather')

## FailedBert

In [10]:
failedbert = pd.read_feather('Data/tickets/failedBERT_latlon.feather')

In [11]:
# create a loc_id based on unique lat-lon combinations in the dataset - to use later for joining

failedbert['lat-lon']=failedbert['lon'].astype(str)+failedbert['lat'].astype(str)
loc_dict = dict(zip(failedbert['lat-lon'].unique().tolist(),range(len(failedbert['lat-lon'].unique()))))
failedbert['fb_loc_id']=failedbert['lat-lon'].map(loc_dict)

In [12]:
failedbert['fb_loc_id'].unique()

array([   0,    1,    2, ..., 9228, 9229, 9230])

In [13]:
%%time

# create table in datebase 

failedbert.to_sql('failedbert',con,if_exists='replace',index=False)

/Users/stuartong/opt/anaconda3/lib/python3.7/site-packages/pandas/core/generic.py:2789: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


CPU times: user 1.94 s, sys: 190 ms, total: 2.13 s
Wall time: 2.2 s


In [14]:
# Then add new column to store geometry

con.enable_load_extension(True)
con.load_extension('mod_spatialite')
con.execute('SELECT InitSpatialMetaData(1);')
con.execute(
    '''
    SELECT AddGeometryColumn('failedbert','wkb_geometry',4326,'POINT',2);
    '''
)

In [15]:
# convert dataframe to GeoDataFrame to get Point Geom from Lat Lon columns

gdf = gpd.GeoDataFrame(failedbert,geometry=gpd.points_from_xy(failedbert.lon,failedbert.lat))

In [16]:
%%time

# Convert each shapely geometry into WKT representation

records = [
    {'Summons Number': gdf['Summons Number'].iloc[i],'wkb':shapely.wkt.dumps(gdf['geometry'].iloc[i])}
    for i in range(gdf.shape[0])
]

CPU times: user 5.5 s, sys: 7.23 ms, total: 5.51 s
Wall time: 5.52 s


In [17]:
# Create tuple of tuples for query parameter (for batch update with executemany)

tuples = tuple((d['wkb'],d['Summons Number'].astype(str)) for d in records)

In [18]:
con.execute("CREATE INDEX  fb_loc_id ON failedbert(fb_loc_id)")
con.execute("CREATE INDEX fb_summons_number ON failedbert(`Summons Number`)")

In [19]:
%%time

# Update with geometry
with sqlite3.connect(db_name) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.executemany(
        """
        UPDATE failedbert
        SET wkb_geometry=ST_PointFromText(? , 4326)
        WHERE failedbert.`Summons Number` = ?;
        """, tuples
    )



CPU times: user 4.23 s, sys: 5.59 s, total: 9.83 s
Wall time: 10 s


In [20]:
# Create a spatial index
spatcon.execute("SELECT CreateSpatialIndex('failedbert','wkb_geometry')")

In [21]:
%%time

#check to ensure KNN is working

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'failedbert'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)
test.head(100)

CPU times: user 7.64 s, sys: 4.86 s, total: 12.5 s
Wall time: 13.1 s


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,failedbert,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,3797,0.000000


## Create new table from unique lat-lons

In [22]:
%%time

spatcon.execute('DROP TABLE IF EXISTS failedbert_points')
spatcon.execute(
    '''
    CREATE TABLE IF NOT EXISTS failedbert_points AS
    Select DISTINCT fb_loc_id,wkb_geometry, count(*) as count
    FROM failedbert
    Group by fb_loc_id
    Order by count DESC
    '''
)
spatcon.execute("SELECT RecoverGeometryColumn('failedbert_points','wkb_geometry',4326,'POINT',2)")
spatcon.execute("SELECT CreateSpatialIndex('failedbert_points','wkb_geometry')")

CPU times: user 116 ms, sys: 34.6 ms, total: 151 ms
Wall time: 258 ms


In [23]:
%%time

# check to ensure spatial index is loaded properly on failedbert_points

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'failedbert_points'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)

test

CPU times: user 176 ms, sys: 3.37 ms, total: 179 ms
Wall time: 180 ms


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,failedbert_points,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,4200,0.000000


## Run KNN on FailedBert Locations to get nearest segment

In [3]:
%%time

query='''
Select k.*,p.* 
FROM knn k, failedbert_points p
WHERE f_table_name = 'LION'
AND ref_geometry = p.wkb_geometry
'''

failedbert_knn=pd.read_sql_query(query,spatcon)
failedbert_knn=failedbert_knn[failedbert_knn['pos']==1][['fid','distance','fb_loc_id']]

CPU times: user 5h 21min 55s, sys: 13min 51s, total: 5h 35min 47s
Wall time: 5h 38min 32s


In [4]:
failedbert_knn

,fid,distance,fb_loc_id
0,105877,30.160454,56
3,89147,0.105165,57
6,26599,0.592041,338
9,25952,2.264506,112
12,80050,2.342688,420
...,...,...,...
27675,58128,22.400715,9226
27678,226468,4108970.987794,9227
27681,64093,0.428283,9228
27684,33788,276078.529343,9229


In [5]:
# create new table
failedbert_knn.to_sql('failedbert_knn',con,if_exists='replace',index=False)

In [6]:
%%time

con.execute('DROP TABLE IF EXISTS failedbert_matched')
con.execute(
    '''
    CREATE TABLE IF NOT EXISTS failedbert_matched AS
    Select a.`Summons Number`,a.`Violation Code`,a.`Issue Date`,a.`Violation Time`,b.*
    FROM failedbert a
    LEFT OUTER JOIN failedbert_knn b
    ON a.fb_loc_id=b.fb_loc_id
    '''
)

CPU times: user 117 ms, sys: 131 ms, total: 248 ms
Wall time: 914 ms


In [7]:
%%time

con.execute('DROP TABLE IF EXISTS failedbert_final')
con.execute(
    '''
    CREATE TABLE IF NOT EXISTS failedbert_final AS
    Select a.`Summons Number`,a.`Violation Code`,a.`Issue Date`,a.`Violation Time`,
    b.OBJECTID,b.Street,b.FeatureTyp,b.SegmentTyp,b.NonPed,b.TrafDir,b.LocStatus,b.LZip,b.RZip,b.LBoro,b.RBoro,
    b.L_CD,b.R_CD,b.CurveFlag,b.Radius,b.RW_Type,b.PhysicalID,b.StreetWidt,b.BikeLane,b.BIKE_Trafd,b.Number_Tra,
    b.Number_Par,b.Number_Tot,b.Posted_Spe,b.Truck_Rout
    FROM failedbert_matched a
    LEFT OUTER JOIN LION b
    ON a.fid=b.OBJECTID
    '''
)

CPU times: user 233 ms, sys: 99 ms, total: 332 ms
Wall time: 455 ms


In [8]:
%%time

query='''
Select *
FROM failedbert_final
'''

test=pd.read_sql_query(query,spatcon)

CPU times: user 1.26 s, sys: 51.9 ms, total: 1.32 s
Wall time: 1.32 s


In [9]:
test.to_feather('Data/tickets/failedbert_matched.feather')

In [10]:
test

,Summons Number,Violation Code,Issue Date,Violation Time,OBJECTID,Street,FeatureTyp,SegmentTyp,NonPed,TrafDir,LocStatus,LZip,RZip,LBoro,RBoro,L_CD,R_CD,CurveFlag,Radius,RW_TYPE,PhysicalID,StreetWidt,BikeLane,BIKE_TRAFD,Number_Tra,Number_Par,Number_Tot,POSTED_SPE,TRUCK_ROUT
0,1334903323,46,09/06/2020,0930A,159260.000000,JAMAICA AVENUE,0,U,None,T,None,11432,11432,4.000000,4.000000,412,412,None,0.000000,1,6423.000000,45.000000,None,None,2,None,4,25,None
1,1447748890,98,01/11/2020,0957A,NaN,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,None,None
2,1448992552,14,01/04/2020,0806P,209721.000000,HYLAN BOULEVARD,0,R,None,W,None,10308,10308,5.000000,5.000000,503,503,None,0.000000,1,15542.000000,32.000000,None,None,3,None,3,35,2
3,1447346210,40,04/30/2020,1225P,NaN,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,None,None
4,1451318250,98,01/05/2020,0837A,91093.000000,EAST 68 STREET,0,U,None,W,None,10065,10065,1.000000,1.000000,108,108,None,0.000000,1,73522.000000,30.000000,None,None,1,None,3,25,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126314,1452723849,40,12/31/2019 12:00:00 AM,0315P,132683.000000,101 AVENUE,0,U,None,T,None,11419,11419,4.000000,4.000000,409,409,None,0.000000,1,4621.000000,44.000000,None,None,2,None,3,25,None
126315,1435549004,70,12/31/2019 12:00:00 AM,0249P,227370.000000,NY-NJ BOUNDARY,3,U,None,None,9,None,None,NaN,5.000000,None,503,None,0.000000,None,NaN,NaN,None,None,None,None,None,None,None
126316,1449060810,14,12/31/2019 12:00:00 AM,0630A,NaN,None,None,None,None,None,None,None,None,NaN,NaN,None,None,None,NaN,None,NaN,NaN,None,None,None,None,None,None,None
126317,1458075382,98,10/16/2019,0700P,88980.000000,HENRY STREET,0,U,None,W,None,10002,10002,1.000000,1.000000,103,103,None,0.000000,1,4249.000000,30.000000,None,None,1,None,3,25,None


In [13]:
np.sort(test['Violation Code'].unique())

array([ 0,  1,  2,  3,  4,  5,  6,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 26, 27, 29, 30, 31, 32, 33, 34, 35, 37, 38,
       39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 56,
       58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 89, 90, 91, 92,
       94, 95, 96, 97, 98, 99])

## Collisions

In [14]:
collision = pd.read_feather('Data/final_collision_updatedlatlon.feather')

In [17]:
collision.head()

,crash_date,crash_time,borough,zip_code,lat,lon,on_street_name,cross_street_name,off_street_name,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,contributing_factor_vehicle_1,contributing_factor_vehicle_2,contributing_factor_vehicle_3,contributing_factor_vehicle_4,contributing_factor_vehicle_5,collision_id,vehicle_type_code_1,vehicle_type_code_2,vehicle_type_code_3,vehicle_type_code_4,vehicle_type_code_5,month,year,on_street_clean,cross_street_clean,off_street_clean,boro_code,the_geom,cartodb_id,date_time,crash_count,contributing_factors,vehicle_types,StreetNameGeo
0,2020-12-11,20:40,None,0,40.741074,-73.725800,CROSS ISLAND PARKWAY,UNION TURNPIKE,None,0.000000,0.000000,0,0,0,0,0,0,Unspecified,Unspecified,None,None,None,4375244,Sedan,Sedan,None,None,None,12,2020,CROSS ISLAND PARKWAY,UNION TURNPIKE,None,NaN,None,NaN,None,NaN,None,None,None
1,2020-12-23,18:03,BROOKLYN,11219,40.637054,-73.986430,14 AVENUE,43 STREET,None,1.000000,0.000000,1,0,0,0,0,0,Driver Inattention/Distraction,None,None,None,None,4380252,Station Wagon/Sport Utility Vehicle,None,None,None,None,12,2020,14 AVENUE,43 STREET,None,3.000000,0101000020E6100000C5724BAB217F52C09BE447FC8A51...,3131234.000000,2020-12-23 18:03:00+00:00,1.000000,Driver Inattention/Distraction,Station Wagon/Sport Utility Vehicle,None
2,2020-12-05,18:42,None,0,40.801754,-73.931210,1 AVENUE,EAST 125 STREET,None,0.000000,0.000000,0,0,0,0,0,0,Passing or Lane Usage Improper,Unspecified,None,None,None,4373655,Station Wagon/Sport Utility Vehicle,Sedan,None,None,None,12,2020,1 AVENUE,EAST 125 STREET,None,NaN,None,NaN,None,NaN,None,None,None
3,2020-12-08,12:25,BROOKLYN,11203,40.653324,-73.938290,LINDEN BOULEVARD,EAST 42 STREET,None,0.000000,0.000000,0,0,0,0,0,0,Driver Inattention/Distraction,Unspecified,None,None,None,4374467,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,None,12,2020,LINDEN BOULEVARD,EAST 42 STREET,None,NaN,None,NaN,None,NaN,None,None,None
4,2020-12-19,16:09,QUEENS,11416,40.684597,-73.844820,None,None,96-04 101 AVENUE,0.000000,0.000000,0,0,0,0,0,0,Passing Too Closely,Unspecified,None,None,None,4377792,Sedan,Sedan,None,None,None,12,2020,None,None,96-04 101 AVENUE,NaN,None,NaN,None,NaN,None,None,None


In [18]:
collision['lat-lon']=collision['lon'].astype(str)+collision['lat'].astype(str)
loc_dict = dict(zip(collision['lat-lon'].unique().tolist(),range(len(collision['lat-lon'].unique()))))
collision['col_loc_id']=collision['lat-lon'].map(loc_dict)

In [19]:
collision['col_loc_id'].unique()

array([    0,     1,     2, ..., 89459, 89460, 89461])

In [21]:
# convert dataframe to GeoDataFrame to get Point Geom from Lat Lon columns

gdf = gpd.GeoDataFrame(collision,geometry=gpd.points_from_xy(collision.lon,collision.lat))

In [22]:
%%time

# Convert each shapely geometry into WKT representation

records = [
    {'collision_id': gdf['collision_id'].iloc[i],'wkb':shapely.wkt.dumps(gdf['geometry'].iloc[i])}
    for i in range(gdf.shape[0])
]

CPU times: user 13.4 s, sys: 75.5 ms, total: 13.5 s
Wall time: 13.5 s


In [23]:
# Create tuple of tuples for query parameter (for batch update with executemany)

tuples = tuple((d['wkb'],d['collision_id'].astype(str)) for d in records)

In [24]:
%%time

# create table in datebase 

collision.drop(['geometry'],axis=1).to_sql('collision',con,if_exists='replace',index=False)

CPU times: user 6.18 s, sys: 297 ms, total: 6.47 s
Wall time: 6.87 s


In [25]:
# Then add new column to store geometry

con.enable_load_extension(True)
con.load_extension('mod_spatialite')
con.execute('SELECT InitSpatialMetaData(1);')
con.execute(
    '''
    SELECT AddGeometryColumn('collision','wkb_geometry',4326,'POINT',2);
    '''
)

In [26]:
con.execute("CREATE INDEX  col_loc_id ON collision(col_loc_id)")
con.execute("CREATE INDEX collision_id ON collision(collision_id)")

In [27]:
%%time

# Update with geometry
with sqlite3.connect(db_name) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.executemany(
        """
        UPDATE collision
        SET wkb_geometry=ST_PointFromText(? , 4326)
        WHERE collision.collision_id = ?;
        """, tuples
    )


CPU times: user 10.5 s, sys: 1min 10s, total: 1min 21s
Wall time: 1min 22s


In [28]:
# Create a spatial index
spatcon.execute("SELECT CreateSpatialIndex('collision','wkb_geometry')")

In [29]:
%%time

#check to ensure KNN is working

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'collision'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)
test.head(100)

CPU times: user 5.96 s, sys: 495 ms, total: 6.45 s
Wall time: 6.5 s


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,collision,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,260893,55.061695


In [30]:
%%time

# get unique locations

spatcon.execute('DROP TABLE IF EXISTS collision_points')
spatcon.execute(
    '''
    CREATE TABLE IF NOT EXISTS collision_points AS
    Select DISTINCT col_loc_id,wkb_geometry, count(*) as count
    FROM collision
    Group by col_loc_id
    Order by count DESC
    '''
)
spatcon.execute("SELECT RecoverGeometryColumn('collision_points','wkb_geometry',4326,'POINT',2)")
spatcon.execute("SELECT CreateSpatialIndex('collision_points','wkb_geometry')")

CPU times: user 1.19 s, sys: 242 ms, total: 1.43 s
Wall time: 1.53 s


In [2]:
%%time

# check to ensure spatial index is loaded properly on collision_points

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'collision_points'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)

test

CPU times: user 81.7 ms, sys: 31.1 ms, total: 113 ms
Wall time: 394 ms


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,collision_points,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,7005,55.061695


In [3]:
%%time
# run knn on unique collision points

query='''
Select k.*,p.* 
FROM knn k, collision_points p
WHERE f_table_name = 'LION'
AND ref_geometry = p.wkb_geometry
'''

collision_knn=pd.read_sql_query(query,spatcon)
collision_knn

CPU times: user 46min 17s, sys: 1min 52s, total: 48min 10s
Wall time: 48min 29s


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance,col_loc_id,wkb_geometry,count
0,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,3,1,162567,8642547.704310,15438,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,456
1,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,3,2,192934,8642547.704310,15438,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,456
2,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,3,3,193016,8642547.704310,15438,b'\x00\x01\xe6\x10\x00\x00\x00\x00\x00\x00\x00...,456
3,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x9f\xc8\x93\xa4kzR\...,3,1,13183,0.270365,475,b'\x00\x01\xe6\x10\x00\x00\x9f\xc8\x93\xa4kzR\...,257
4,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x9f\xc8\x93\xa4kzR\...,3,2,15118,2.753825,475,b'\x00\x01\xe6\x10\x00\x00\x9f\xc8\x93\xa4kzR\...,257
...,...,...,...,...,...,...,...,...,...,...
268381,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xf3u\x19\xfe\xd3mR\...,3,2,133096,1.004340,89460,b'\x00\x01\xe6\x10\x00\x00\xf3u\x19\xfe\xd3mR\...,1
268382,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xf3u\x19\xfe\xd3mR\...,3,3,154049,1.159317,89460,b'\x00\x01\xe6\x10\x00\x00\xf3u\x19\xfe\xd3mR\...,1
268383,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xd2\xc7|@\xa0yR\xc0...,3,1,31083,0.616486,89461,b'\x00\x01\xe6\x10\x00\x00\xd2\xc7|@\xa0yR\xc0...,1
268384,lion,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xd2\xc7|@\xa0yR\xc0...,3,2,20123,75.406422,89461,b'\x00\x01\xe6\x10\x00\x00\xd2\xc7|@\xa0yR\xc0...,1


In [4]:
# ran above at 11:11am Sat May 15

In [5]:
collision_knn=collision_knn[collision_knn['pos']==1][['fid','distance','col_loc_id']]
collision_knn

,fid,distance,col_loc_id
0,162567,8642547.704310,15438
3,13183,0.270365,475
6,48975,0.269969,2296
9,7528,0.680239,3165
12,1140,0.361820,51
...,...,...,...
268371,217591,0.097036,89457
268374,152023,0.382789,89458
268377,6203,1.173892,89459
268380,141937,0.448269,89460


In [6]:
# create new table
collision_knn.to_sql('collision_knn',con,if_exists='replace',index=False)

In [7]:
%%time

con.execute('DROP TABLE IF EXISTS collision_matched')
con.execute(
    '''
    CREATE TABLE IF NOT EXISTS collision_matched AS
    Select a.crash_date,a.crash_time,a.collision_id,a.number_of_persons_injured,a.number_of_persons_killed,a.number_of_pedestrians_injured,a.number_of_pedestrians_killed,
    a.number_of_cyclist_injured,a.number_of_cyclist_killed,a.number_of_motorist_injured,a.number_of_motorist_killed,b.*
    FROM collision a
    LEFT OUTER JOIN collision_knn b
    ON a.col_loc_id=b.col_loc_id
    '''
)

CPU times: user 483 ms, sys: 381 ms, total: 864 ms
Wall time: 1.68 s


In [8]:
%%time

# check to ensure spatial index is loaded properly on collision_points

query='''
Select * 
FROM collision_matched
'''

test=pd.read_sql_query(query,spatcon)

test

CPU times: user 1.5 s, sys: 60.1 ms, total: 1.56 s
Wall time: 1.57 s


,crash_date,crash_time,collision_id,number_of_persons_injured,number_of_persons_killed,number_of_pedestrians_injured,number_of_pedestrians_killed,number_of_cyclist_injured,number_of_cyclist_killed,number_of_motorist_injured,number_of_motorist_killed,fid,distance,col_loc_id
0,2020-12-11,20:40,4375244,0.000000,0.000000,0,0,0,0,0,0,170336,0.818134,0
1,2020-12-23,18:03,4380252,1.000000,0.000000,1,0,0,0,0,0,42726,0.134661,1
2,2020-12-05,18:42,4373655,0.000000,0.000000,0,0,0,0,0,0,96793,0.041010,2
3,2020-12-08,12:25,4374467,0.000000,0.000000,0,0,0,0,0,0,58278,0.057493,3
4,2020-12-19,16:09,4377792,0.000000,0.000000,0,0,0,0,0,0,125034,1.636027,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298066,2019-01-02,10:00,4060794,0.000000,0.000000,0,0,0,0,0,0,39751,0.205249,66147
298067,2019-01-01,18:20,4062967,0.000000,0.000000,0,0,0,0,0,0,50762,0.066175,16336
298068,2019-01-03,3:45,4060999,0.000000,0.000000,0,0,0,0,0,0,31083,0.616486,89461
298069,2019-01-01,2:25,4060746,2.000000,0.000000,0,0,0,0,2,0,68186,0.742612,16030


In [9]:
test.to_feather('Data/collision_nozip_matched.feather')

## Getting Mid Point for Each Object In LION
For mapping

In [7]:
query='''
Select OBJECTID, ST_AsText(LINE_INTERPOLATE_POINT(LION.wkb_geometry,0.5)) as mid_point
FROM LION
'''

midpoints=pd.read_sql_query(query,spatcon)

midpoints.to_feather('Data/segmentmidpoints.feather')

In [8]:
midpoints

,OBJECTID,mid_point
0,1,POINT(-73.902925 40.830008)
1,2,POINT(-73.901638 40.866814)
2,3,POINT(-73.900842 40.825207)
3,4,POINT(-73.900842 40.825207)
4,5,POINT(-73.900842 40.825207)
...,...,...
230553,230554,POINT(-74.245071 40.515552)
230554,230555,POINT(-74.074617 40.642321)
230555,230556,POINT(-74.074642 40.642247)
230556,230557,POINT(-74.074662 40.642312)


## Get Closest Weather Station for Each Street Segment 

In [9]:
stations = pd.read_feather('Data\w_stations_loc.feather')

In [10]:
stations = gpd.GeoDataFrame(stations,geometry=gpd.points_from_xy(stations.LONGITUDE,stations.LATITUDE))

In [31]:
stations['station_id']= range(1,len(stations)+1)
stations

,STATION,LATITUDE,LONGITUDE,geometry,station_id
0,US1CTFR0039,41.037788,-73.568176,POINT (-73.56818 41.03779),1
1,US1NJBG0003,40.914670,-73.977500,POINT (-73.97750 40.91467),2
2,US1NJBG0010,40.991450,-74.012348,POINT (-74.01235 40.99145),3
3,US1NJBG0015,40.791492,-74.139790,POINT (-74.13979 40.79149),4
4,US1NJBG0017,40.951090,-74.118264,POINT (-74.11826 40.95109),5
...,...,...,...,...,...
129,USW00054787,40.734170,-73.416940,POINT (-73.41694 40.73417),130
130,USW00094728,40.778980,-73.969250,POINT (-73.96925 40.77898),131
131,USW00094741,40.850000,-74.061390,POINT (-74.06139 40.85000),132
132,USW00094745,41.062360,-73.704630,POINT (-73.70463 41.06236),133


In [32]:
%%time

# create table in datebase 

stations.drop(['geometry'],axis=1).to_sql('stations',con,if_exists='replace',index=False)

CPU times: user 8.51 ms, sys: 6.8 ms, total: 15.3 ms
Wall time: 105 ms


In [13]:
# Then add new column to store geometry

con.enable_load_extension(True)
con.load_extension('mod_spatialite')
con.execute('SELECT InitSpatialMetaData(1);')
con.execute(
    '''
    SELECT AddGeometryColumn('stations','wkb_geometry',4326,'POINT',2);
    '''
)

In [15]:
con.execute("CREATE INDEX station_id ON stations(STATION)")

In [18]:
%%time

# Convert each shapely geometry into WKT representation

records = [
    {'station': stations['STATION'].iloc[i],'wkb':shapely.wkt.dumps(stations['geometry'].iloc[i])}
    for i in range(stations.shape[0])
]

# Create tuple of tuples for query parameter (for batch update with executemany)

tuples = tuple((d['wkb'],d['station']) for d in records)

CPU times: user 7.91 ms, sys: 1.41 ms, total: 9.31 ms
Wall time: 8.14 ms


In [20]:
%%time

# Update with geometry
with sqlite3.connect(db_name) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.executemany(
        """
        UPDATE stations
        SET wkb_geometry=ST_PointFromText(? , 4326)
        WHERE stations.STATION = ?;
        """, tuples
    )


CPU times: user 9.5 ms, sys: 10.3 ms, total: 19.8 ms
Wall time: 201 ms


In [21]:
# Create a spatial index
spatcon.execute("SELECT CreateSpatialIndex('stations','wkb_geometry')")

In [22]:
%%time

#check to ensure KNN is working

query='''
Select k.* 
FROM knn k 
WHERE f_table_name = 'stations'
AND ref_geometry = MakePoint(-73.9215944000000036, 40.8278258999999935)
AND max_items = 1
'''

test=pd.read_sql_query(query,spatcon)
test.head(100)

CPU times: user 5.42 ms, sys: 3.78 ms, total: 9.2 ms
Wall time: 106 ms


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance
0,stations,wkb_geometry,b'\x00\x01\x00\x00\x00\x00P\x0b\x14g\xfbzR\xc0...,1,1,97,3562.490943


### For each street segment, find nearest weather station using KNN

In [24]:
%%time

query='''
Select k.* ,l.OBJECTID
FROM knn k, LION l 
WHERE f_table_name = 'stations'
AND ref_geometry = l.wkb_geometry
'''

test=pd.read_sql_query(query,spatcon)
test.head(100)

CPU times: user 11min 1s, sys: 2.48 s, total: 11min 4s
Wall time: 11min 20s


,f_table_name,f_geometry_column,ref_geometry,max_items,pos,fid,distance,OBJECTID
0,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xf7\x82\xa0l\xd2yR\...,3,1,97,4469.360816,1
1,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xf7\x82\xa0l\xd2yR\...,3,2,77,5839.743051,1
2,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xf7\x82\xa0l\xd2yR\...,3,3,127,5878.106284,1
3,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x8b\xb8\xe4\x97\xbb...,3,1,77,4739.782961,2
4,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\x8b\xb8\xe4\x97\xbb...,3,2,97,8204.737802,2
...,...,...,...,...,...,...,...,...
95,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xed\t\xc2r\xeeyR\xc...,3,3,11,7206.952649,32
96,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xa8>\xc0\xe9\xf2yR\...,3,1,77,5480.876217,33
97,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xa8>\xc0\xe9\xf2yR\...,3,2,2,7131.703681,33
98,stations,wkb_geometry,b'\x00\x01\xe6\x10\x00\x00\xa8>\xc0\xe9\xf2yR\...,3,3,11,8337.049838,33


In [33]:
lion_knn_weather = test[test['pos']==1]

In [34]:
# create new table
lion_knn_weather.to_sql('lion_knn_weather',con,if_exists='replace',index=False)

In [42]:
%%time

query='''
Select a.OBJECTID, b.STATION,b.station_id,a.distance
FROM lion_knn_weather a
LEFT OUTER JOIN stations b
ON a.fid=b.station_id
'''

nearest_weather_station=pd.read_sql_query(query,spatcon)

CPU times: user 421 ms, sys: 29.9 ms, total: 451 ms
Wall time: 461 ms


In [44]:
nearest_weather_station.to_feather('Data/segmentmatchedtostation.feather')